In [ ]:
import os
import gradio as gr
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain.vectorstores import FAISS
from langchain.vectorstores.base import VectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

OPENAI_MODEL = 'gpt-4o-mini'
GEMINI_MODEL = 'gemini-2.5-flash-preview-05-20'
PERSIST_DB = 'vector_db'

In [ ]:
class QueryYourPDF:
    def __init__(self, model, vector):
        self.model = model
        self.vector = vector
        self.retriever = None
        self.chain = None

    def process_files(self, pdf_files):
        all_doc = []

        for pdf in pdf_files:
            loader = PyPDFLoader(pdf.name)
            loaded_doc = loader.load()
            all_doc.extend(loaded_doc)

        result = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = result.split_documents(all_doc)

        if self.model == 'GPT':
            embeddings = OpenAIEmbeddings()
            llm = ChatOpenAI(temperature=0.7)
        elif self.model == 'Gemini':
            embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
            llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL)
        else:
            raise ValueError(f'Unsupported Model choice {self.model}')

        if self.vector == 'FAISS':
            db = FAISS.from_documents(chunks, embedding=embeddings)
        elif self.vector == 'Chroma':
            if os.path.exists(PERSIST_DB):
                Chroma(persist_directory=PERSIST_DB, embedding_function=embeddings).delete_collection()
            db = Chroma.from_documents(chunks, embedding=embeddings, persist_directory=PERSIST_DB)
            db.persist()

        retriever = db.as_retriever()
        memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

        self.chain = ConversationalRetrievalChain.from_llm(llm=llm, memory=memory, retriever=retriever)

    def query_kb(self, query, chat_history):
        return self.chain.invoke({'question' :query, 'chat_history' : chat_history})['answer']

    def reset(self):
        self.chain = None
        self.retriever = None

In [ ]:
app_instance = None
chat_history = []

def initialize_app(model_name, vector_name,files):
    global app_instance
    if not files:
        app_instance = None
        return "please upload pdf documents"

    app_instance = QueryYourPDF(model_name, vector_name)
    app_instance.process_files(files)

    return "files upload successful!!!!!"


def process_query(query, chat_history):
    global app_instance
    if app_instance == None:
        return "pleasee upload pdf documents", chat_history

    response = app_instance.query_kb(query, chat_history)

    chat_history.append({'role' : 'user', 'content' : query})
    chat_history.append({'role' : 'assistant', 'content' : response})

    return '', chat_history

def reset_app():
    global app_instance, chat_history
    if app_instance:
        app_instance.reset()
    app_instance = None
    chat_history = []
    return None, '', [], 'please upload pdf documents'

In [ ]:
with gr.Blocks() as app:
    gr.Markdown('Upload PDF Documents:')
    
    with gr.Row():
        model_choice = gr.Dropdown(value='GPT', label='Model:', choices=['GPT', 'Gemini'])
        vector_choice = gr.Dropdown(value='FAISS', label='Vector:', choices=['FAISS', 'Chroma'])
    
    file_input = gr.File(file_count='multiple', file_types=['.pdf'], label='File Upload:')
    init_status = gr.Textbox(interactive=False, label='Status:')
    chatbot = gr.Chatbot(type='messages', label='Chat:')
    query = gr.Textbox(label='Ask a question:')
    
    with gr.Row():
        submit = gr.Button('Submit')
        reset = gr.Button('Reset')
        
    file_input.change(
        fn=initialize_app,
        inputs=[model_choice, vector_choice, file_input],
        outputs=init_status
    )

    submit.click(
        fn=process_query,
        inputs=[query, chatbot],
        outputs=[query, chatbot]
    )

    reset.click(
        fn=reset_app, 
        outputs=[file_input, query, chatbot, init_status]
    )
    

In [ ]:
app.launch()